In [ ]:
import sympy
from IPython.display import Latex, display
from sympy import Poly
from sympy.abc import s, z

from control.matlab import *

In [1]:
%pylab %matplotlib inline

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


<div id="toc"></div>

# Introduction: Simulink Control



## The open-loop plant model

In the [Introduction: Simulink Modeling](../Introduction/Introduction-SimulinkModeling.ipynb) page we demonstrated how Simulink can be employed to simulate a physical system. More generally, Simulink can also simulate the complete control system, including the control algorithm in addition to the physical plant. As mentioned previously, Simulink is especially useful for generating the approximate solutions of mathematical models that may be prohibitively difficult to solve "by hand." For example, consider that you have a nonlinear plant. A common approach is to generate a linear approximation of the plant and then use the linearized model to design a controller using analytical techniques. Simulink can then be employed to simulate the performance of your controller when applied to the full nonlinear model. Simulink can be employed for generating the linearized model and MATLAB can be employed for designing the controller as described in the other Introduction pages. Various control design facilities of MATLAB can also be accessed directly from within Simulink. We will demonstrate both approaches in this page. Recall the Simulink model of the toy train system derived in the [Introduction: Simulink Modeling](../Introduction/Introduction-SimulinkModeling.ipynb) page and pictured below.
![inoutv.png](figures/inoutv.png)
You can generate this model yourself, or you can download the completed model <Content/Introduction/Simulink/Control/train.mdl here>. Assuming that the train only travels in one dimension (along the track), we want to apply control to the train engine so that it starts and comes to rest smoothly, and so that it can track a constant speed command with minimal error in steady state.

## Implementing a PID controller in Simulink

Let us first create the structure for simulating the train system in unity feedback with a PID controller. In order to make our Simulink model more understandable, we will first save the train model into its own subsystem block. To accomplish this, delete the three scope blocks and replace each one by an Out1 block from the Sinks library. Label each Out1 block with the corresponding variable name, "x1_dot", "x1", and "x2". Then delete the Signal Generator block and replace it with an In1 block from the Sources library. Label this input "F" for the force generated between the train engine and the railroad track. Your model should now appear as follows.
![inout2.png](figures/inout2.png)
Next select all of the blocks in your model (**Ctrl A**) and select **Create Subsystem** from the **Edit** menu at the top of the model window. With a little rearranging and relabeling, your model will appear as shown below.
![subsystem.png](figures/subsystem.png)
Now we can add a controller to our system. We will employ a PID controller which can be implemented using a PID Controller block from the Continuous library. Placing this block in series with the train subsystem, your model will appear as follows. In the following, we model the controller as generating the force "F" directly. This neglects the dynamics with which the train engine generates the torque applied to the wheels, and subsequently neglects the dynamics of how the force is generated at the wheel/track interface. This simplified approach is taken at this point since we only wish to introduce the basic functionality of Simulink for controller design and analysis.
![pid.png](figures/pid.png)
Double-clicking on the PID Controller block, we will initially set the *Integral (I)** gain field equal to 0 and will leave the **Proportional (P)** and **Derivative (D)* gains as their defaults of 1 and 0, respectively. Next add a Sum block from the Math Operations library. Double-click on this block and modify the **List of signs** field to "|+-". Since we wish to control the velocity of the toy train engine, we will feedback the engine's velocity. This is accomplished by 4ring a line off of the "x1_dot" signal and connecting it to the negative sign of the Sum block. The output of the Sum block will be the velocity error for the train engine and should be connected to the input of the PID Controller block. Connecting the blocks as described and adding labels, your model should appear as follows.
![feedback.png](figures/feedback.png)
Next add a Signal Builder block from the Sources library to represent the velocity commanded to the train. Since we wish to design a controller to bring the train smoothly up to speed and smoothly to rest, we will test the system with a velocity command that steps up to 1 m/s followed by a step back down to 0 m/s (recall that our system is a _toy_ train). To generate this type of command signal, double-click on the Signal Builder block. Then choose **Change time range** from the **Axes** menu at the top of the block's dialog window. Set the **Max time** field to "300" seconds. Next, set the step up to occur at 10 seconds and the step down to occur at 150 seconds. This is accomplished by clicking on the corresponding portions of the signal graph (left and right vertical lines) and either dragging the line to the desired position, or entering the desired time in the **T** field at the bottom of the window. When done, your signal should appear as follows.
![signal.png](figures/signal.png)
Also add a Scope block from the Sinks library and use it to replace the Out1 block for the train's velocity. Relabeling the blocks, your model will appear as follows.
![train_cl.png](figures/train_cl.png)
We are now ready to run the closed-loop simulation. If you wish to skip the above steps, you may download the completed model with control <Content/Introduction/Simulink/Control/train_cl.mdl here>.

In [2]:
#


## Running the closed-loop model

Before running the model, we need to assign numerical values to each of the variables used in the model. For the train system, we will employ the following values.
* \\(M_1\\) = 1 kg
* \\(M_2\\) = 0.5 kg
* \\(k\\) = 1 N/sec
* \\(F\\) = 1 N
* \\(\mu\\) = 0.02 sec/m
* \\(g\\) = 9.8 m/s^2 Create a new [m-file](../Extras_Mfile.ipynb) and enter the following commands.

In [3]:
# M1 = 1;
# M2 = 0.5;
# k  = 1;
# F  = 1;
# mu = 0.02;
# g  = 9.8;


Execute your m-file in the MATLAB command window to define these values. Simulink will recognize these MATLAB variables for use in the model. Next we need to set the time for which our simulation will run to match the time range of the command from the Signal Builder block. This is accomplished by selecting **Parameters** from the **Simulation** menu at the top of the model window and changing the **Stop Time** field to "300". Now, run the simulation and open the "x1_dot" scope to examine the velocity output (hit autoscale). The result as shown below demonstrates that the closed-loop system is unstable for this controller.
![x1_dot_cl.png](figures/x1_dot_cl.png)
Since the performance achieved above is unsatisfactory, we need to redesign our controller. We will first demonstrate how to extract a model from Simulink into MATLAB for analysis and design. Then we will demonstrate how to design the control from directly within Simulink.

In [4]:
#


In [5]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>